The projections I pulled from ESPN are in a terrible state - pull them into a DataFrame for viz purposes.

See Notion for deets.

In [59]:
import re
import pandas as pd

from typing import List, Dict

In [60]:
pd.set_option("display.max_rows", 1000)

In [61]:
PROJECTIONS_FILE_PATH = "./raw_projections.txt"

In [62]:
with open(PROJECTIONS_FILE_PATH) as flines:
    data = [line.strip() for line in flines]

name three times, categories, then 2020 statistics, then 2021 projections, then some cruft, then repeat

In [63]:
player = []
players = []
for line in data:
    if line.endswith("Headshot"):
        players.append(player)
        player = []
    player.append(line)
players.append(player)  # don't forget the last player

In [64]:
players = players[1:]

In [65]:
def parse_player(input_player: List[str]) -> Dict:
    # Name is {} Headshot
    player = dict()
    name = re.match("^.*(?=Headshot)", input_player[0])[0].strip()
    assert input_player[0].startswith(name),name
    assert input_player[1].startswith(name), input_player[1]
#     assert input_player[2].startswith(name), name
    # position is the 3rd line
    positions = list(re.findall(r"C(?=$|,)|SF|PF|PG|SG", input_player[2]))
#     print(positions)
    remainder = [line for line in input_player[5:] if line]
#     print(remainder)
    categories = remainder[:12]
    stats_2020 = remainder[13:25]
    projections_2021 = remainder[26:38]
    assert categories == ['GP', 'MIN', 'FG%', 'FT%', '3PM', 'REB', 'AST', 'A/TO', 'STL', 'BLK', 'TO', 'PTS']
    assert len(stats_2020) == len(categories) == len(projections_2021)
    return {"name": name,
            "positions": positions,
            "2020": dict(zip(categories, stats_2020)),
            "2021": dict(zip(categories, projections_2021)),
           }
        

In [66]:
len(players)

150

In [67]:
player_df = pd.DataFrame([parse_player(x) for x in players])

In [68]:
projections = pd.concat([player_df.drop(columns=["2020", "2021"]), pd.json_normalize(player_df["2021"])], axis=1)
stats = pd.concat([player_df.drop(columns=["2020", "2021"]), pd.json_normalize(player_df["2020"])], axis=1)


In [69]:
stats

,name,positions,GP,MIN,FG%,FT%,3PM,REB,AST,A/TO,STL,BLK,TO,PTS
0,Nikola Jokic,[C],72,34.6,.566,.868,1.3,10.8,8.3,2.69,1.3,0.7,3.1,26.4
1,Giannis Antetokounmpo,[PF],61,33.0,.569,.685,1.1,11.0,5.9,1.72,1.2,1.2,3.4,28.1
2,Luka Doncic,[PG],66,34.3,.479,.730,2.9,8.0,8.6,2.02,1.0,0.5,4.3,27.7
3,James Harden,"[SG, PG]",44,36.5,.466,.861,2.8,7.9,10.8,2.68,1.2,0.8,4.0,24.6
4,Karl-Anthony Towns,[C],50,33.8,.486,.859,2.4,10.6,4.5,1.41,0.8,1.1,3.2,24.8
5,Bradley Beal,[SG],60,35.7,.485,.889,2.2,4.7,4.4,1.42,1.2,0.4,3.1,31.3
6,Damian Lillard,[PG],67,35.8,.451,.927,4.1,4.2,7.5,2.49,0.9,0.3,3.0,28.7
7,Paul George,"[SF, SG]",54,33.8,.467,.868,3.2,6.6,5.2,1.56,1.1,0.4,3.3,23.3
8,Stephen Curry,[PG],63,34.2,.482,.916,5.3,5.5,5.8,1.70,1.2,0.1,3.4,32.0
9,Jayson Tatum,"[PF, SF]",64,35.8,.459,.868,2.9,7.4,4.3,1.60,1.2,0.5,2.7,26.4


In [70]:
projections

,name,positions,GP,MIN,FG%,FT%,3PM,REB,AST,A/TO,STL,BLK,TO,PTS
0,Nikola Jokic,[C],78,35.1,.561,.862,1.3,11.1,8.7,2.72,1.3,0.7,3.2,26.1
1,Giannis Antetokounmpo,[PF],76,33.3,.564,.717,1.2,12.4,6.0,1.66,1.2,1.2,3.6,29.7
2,Luka Doncic,[PG],77,34.9,.475,.732,3.1,8.3,8.7,2.02,1.0,0.5,4.3,28.3
3,James Harden,"[SG, PG]",76,34.4,.458,.863,3.0,7.2,9.5,2.44,1.3,0.7,3.9,25.2
4,Karl-Anthony Towns,[C],76,33.7,.508,.814,2.8,11.2,4.1,1.32,0.9,1.3,3.1,25.8
5,Bradley Beal,[SG],76,36.8,.471,.864,2.9,4.6,5.3,1.61,1.2,0.4,3.3,33.6
6,Damian Lillard,[PG],76,35.1,.451,.914,3.8,4.2,7.7,2.66,0.9,0.3,2.9,28.9
7,Paul George,"[SF, SG]",76,36.5,.440,.841,3.7,8.0,4.2,1.56,2.1,0.4,2.7,27.5
8,Stephen Curry,[PG],73,33.9,.477,.916,5.2,5.4,5.5,1.78,1.3,0.2,3.1,29.5
9,Jayson Tatum,"[PF, SF]",76,36.1,.463,.860,3.1,7.4,4.1,1.58,1.2,0.6,2.6,28.8


In [71]:
len(projections)

150

In [72]:
stats.to_csv("stats.csv", index=False)

In [73]:
projections.to_csv("projections.csv", index=False)